In [1]:
%reload_ext Cython
%reload_ext autoreload
%reload_ext line_profiler

In [2]:
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c, e, m_p
from scipy.signal import fftconvolve

%matplotlib tk
import seaborn as sns
sns.set_context('talk', font_scale=1.5,
                rc={'lines.markeredgewidth': .1})
sns.set_style('darkgrid', {
        'axes.linewidth': 2,
        'legend.fancybox': True})

/usr/lib64/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
from PyHEADTAIL.trackers.rf_bucket import RFBucket
import PyHEADTAIL.particles.generators as gen

PyHEADTAIL v1.9.4.40




## Description

This script is supposed to test the implementation of multi-turn multi-bunch wakes. It tests in particular the implementation of the convolution which becomes a little tricky due to all the book-keeping involved for the multiple turns and the multiple bunches.

The script employs a simple resonator constant wake field and computes the wake kicks for several beam configurations which are conceptually different but mathematicalle/computationally identical. This is done by first initializing 3 separate bunches with individual bunch lengths (epsn_z) and delays (dz). The bunches are replicated at a distance of one circumference resulting in $2\times 3 = 6$ bunches in total. Three cases are then studied and compared:

1. All bunches are added into one total beam. A long slicer is used to compute the standard single bunch wake kick. This is the original computation and no book-keeping is involved. The bunch-to-bunch wake should be visible in the first half of the computed wake kick. The turn-by-turn wakes should be visible in the later half of the computed wake kicks. Inbetween, the ringing of the wakes will be seen.
2. Only the 3 initial bunches are added into a beam. The later half of the wake kicks should now evolve from the turn-by-turn computation of the wake fields. We hope to see the wake kicks in the later half to overlap.
3. This is the full granularity of multi-turn multi-bunch wake kick computation whivch involves the full book-keeping. We treat all 3 bunches individually and track them for 2 turns. We hope to see the individual wake kicks of each bunch to overlap in the first half for turn 1 and in the later half for turn 2.

***

### Create bunches and bunches_list

In [4]:
momentum = 450e9 * e/c
gamma = np.sqrt((momentum/(m_p*c))**2 + 1)
beta = np.sqrt(1 - gamma**-2)

circumference = 60
alpha = 3.225e-4
h1 = 20
V1 = 2e3
p_increment = 0 * e/c * circumference/(beta*c)

print gamma

479.606062093


In [5]:
rfbucket = RFBucket(
    charge=e, mass=m_p, gamma=gamma,
    circumference=circumference,
    alpha_array=[alpha], p_increment=0,
    harmonic_list=[h1], voltage_list=[V1], phi_offset_list=[0])

In [6]:
T0 = circumference/beta/c
dt = (0, 5e-9, 12.5e-9, T0+0, T0+5e-9, T0+12.5e-9)
epsn_z = (0.35, 0.25, 0.1, 0.35, 0.25, 0.1)
# epsn_z = (0.1, 0.25, 0.35, 0.1, 0.25, 0.35)

In [7]:
bunches_list = [
    gen.ParticleGenerator(
        macroparticlenumber=1e5, intensity=1e11,
        charge=e, mass=m_p, gamma=gamma,
        circumference=circumference,
        distribution_x=gen.gaussian2D(2e-6),
        distribution_y=gen.gaussian2D(2e-6),
        distribution_z=gen.RF_bucket_distribution(rfbucket, epsn_z=epsn_z[i])
    ).generate() for i in xrange(3)]

bunches_list = [deepcopy(b) for i in range(2) for b in bunches_list]
for i, b in enumerate(bunches_list):
    b.z -= dt[i] * (b.beta*c)
    b.dt = dt[i]

bunches = sum(bunches_list)
bunches.dt = dt[0]

*** Maximum RMS emittance 2.37895476067eV s.
... distance to target emittance: 1.16e-02
... distance to target emittance: 1.18e-02
... distance to target emittance: 1.51e-05
--> Emittance: 0.350000019563
--> Bunch length:0.165598558077

/home/kli/workspace/PyHEADTAIL/particles/generators.py:334: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  coords = [np.random.normal(loc=0., scale=std, size=n_particles),
/home/kli/workspace/PyHEADTAIL/particles/generators.py:335: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  np.random.normal(loc=0., scale=std, size=n_particles)]
/home/kli/workspace/PyHEADTAIL/particles/generators.py:535: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  u = uniform(low=xmin, high=xmax, size=n_gen)



*** Maximum RMS emittance 2.37895476067eV s.
... distance to target emittance: 5.69e-03
... distance to target emittance: 5.82e-03
... distance to target emittance: 3.33e-06
--> Emittance: 0.250000001947
--> Bunch length:0.13928832556
*** Maximum RMS emittance 2.37895476067eV s.
... distance to target emittance: 8.61e-04
... distance to target emittance: 9.73e-04
... distance to target emittance: 7.45e-08
--> Emittance: 0.100000000006
--> Bunch length:0.0875026472416


/home/kli/workspace/PyHEADTAIL/particles/generators.py:536: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  v = uniform(low=ymin, high=ymax, size=n_gen)
/home/kli/workspace/PyHEADTAIL/particles/generators.py:537: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  s = uniform(size=n_gen)


In [8]:
# cmap = sns.color_palette('viridis', 3)
# fig, ax = plt.subplots(figsize=(16,9))

# [ax.scatter(b.z, b.dp, marker='o', edgecolor='k', color=cmap[i]) for i, b in enumerate(bunches_list[:3])]
# ax.set_xlabel(r'$z$ [m]', fontsize=24)
# ax.set_ylabel(r'$\delta$', fontsize=24)
# ax.set_ylim(-1e-3, 1e-3)
# plt.show()

### Wakes and wake kicks and functions

In [9]:
%autoreload 2
from PyHEADTAIL.particles.slicing import UniformBinSlicer
from PyHEADTAIL.impedances.wakes import CircularResonator, WakeField
from PyHEADTAIL.impedances.wake_kicks import ConstantWakeKickX

In [10]:
slicer = UniformBinSlicer(100, z_cuts=(-.5, .5))

class CircularResonatorMod(CircularResonator):
    
    def get_wake_kicks(self, slicer):
        wake_kicks = []
        
        if self.Yokoya_X1:
            wake_function = self.function_transverse(self.Yokoya_X1)
            wake_kicks.append(ConstantWakeKickX(
                wake_function, slicer, self.n_turns_wake))
            
        return wake_kicks
    
wake = CircularResonatorMod(R_shunt=1e6, frequency=700e6, Q=350, n_turns_wake=4)

wakefields = WakeField(slicer, wake, circumference=circumference)
wakekick = wakefields.wake_kicks[0]
wakefunction = wakekick.wake_function

*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!


### Convolutions

In [11]:
def convolution_python(t_target, t_source, c_source, w):
    dxp = 0.*t_target
    for k in xrange(len(t_target)):
        for l in xrange(len(t_source)):
            dxp[k] += c_source[l]*w(t_target[k]-t_source[l])

    return dxp

In [12]:
def convolution_numpy(t_target, t_source, c_source, w):
    tmin, tmax = t_source[0], t_source[-1]
    tt = np.concatenate((t_target-tmax, (t_target-tmin)[1:]))
    
    return np.convolve(c_source, w(tt), mode='valid')

In [13]:
def convolve_multibunch(bunches, times, ages, cmoments, betas, wf, f_convolution=convolution_python):
    dxp = []
    bunches = np.atleast_1d(bunches)
    
    n_turns, n_bunches, n_slices = times.shape
    dt = [b.dt for b in bunches]

    for i in xrange(n_bunches):
        z = 0
        t_target = times[0,i]
        n_bunches_infront = i+1
        for k in xrange(n_turns):
            if k>0:
                n_bunches_infront = n_bunches
            for j in range(n_bunches_infront):
                t_source = times[k,j] + ages[k] + dt[i] - dt[j]
                c_source = cmoments[k,j]
                z += f_convolution(t_target, t_source, c_source, wf) # betas not used here thanks to wake
            
        dxp.append(z)
        
    return dxp

In [14]:
def convolve_PyHEADTAIL_sb(bunch, times, ages, cmoments, betas):
    return wakekick._accumulate_source_signal(bunch, times, ages, cmoments, betas)

In [15]:
def convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas):
    return wakekick._accumulate_source_signal_multibunch(bunches, times, ages, cmoments, betas)

In [16]:
def get_times_and_moments(bunches_list, slicer):
    bunches_list = np.atleast_1d(bunches_list)

    slices_list = []
    times_list = []
    cmoments_list = []
    for i, b in enumerate(bunches_list):
        distance = dt[i]*b.beta*c
        b.z += distance
        slices_list.append(b.get_slices(slicer))
        times_list.append(slices_list[-1].z_centers / (b.beta*c))
        cmoments_list.append(slices_list[-1].charge_per_slice)
        b.z -= distance

    # Multi-turn multi-bunch times and moments arrays - here, only one turn assumed
    n_turns = 1
    n_bunches = len(bunches_list)
    n_slices = len(times_list[0])

    tmp = np.zeros((n_turns, n_bunches, n_slices))
    tmp[0,:,:] = times_list
    times_array = tmp

    tmp = np.zeros((n_turns, n_bunches, n_slices))
    tmp[0,:,:] = cmoments_list
    cmoments_array = tmp

    ages_list = np.array([0 for b in bunches_list])
    betas_list = np.array([[b.beta for b in bunches_list]])
    
    return times_array, ages_list, cmoments_array, betas_list

## Cases

#### 1. All $2\times 3$ bunches concatenated in a linac for the full wake kick computation using a long slicer
---

In [17]:
slicer = UniformBinSlicer(4000, z_cuts=(-65, .5))

times, ages, cmoments, betas = get_times_and_moments(bunches, slicer)

delta_xp_mb = convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas)
delta_xp_np = convolve_multibunch(bunches, times, ages, cmoments, betas, wakefunction)

times_list = times[:,0,:]
cmoments_list = cmoments[:,0,:]

delta_xp_sb = convolve_PyHEADTAIL_sb(bunches, times_list, ages, cmoments_list, betas)

In [18]:
times = np.squeeze(times)
cmoments = np.squeeze(cmoments)
dxp1 = np.squeeze(delta_xp_mb)
dxp2 = np.squeeze(delta_xp_sb)
dxp3 = np.squeeze(delta_xp_np)

t0 = np.concatenate((times-times[-1], (times-times[0])[1:]))
to_z = beta*c

col = sns.color_palette('cubehelix', 3, 0.8)
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(16,10), sharex=True)

ax1.plot(t0*to_z, wakefunction(t0)[::-1]/max(wakefunction(t0)[::-1]))
ax1.plot((times-times[-1])*to_z, cmoments/max(cmoments))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Wake', 'Bunch'], loc=0)
ax1.set_xlim(-66, 6)

ax2.plot((times-times[-1])*to_z, dxp1/max(dxp1))
ax2.plot((times-times[-1])*to_z, dxp2/max(dxp2))
ax2.plot((times-times[-1])*to_z, -dxp3/max(-dxp3))
ax2.legend(['PyHEADTAIL MB', 'PyHEADTAIL SB', 'Notebook manual'], loc=0)

ax3.plot((times-times[-1])*to_z, 2*cmoments/max(cmoments))
ax3.plot((times-times[-1])*to_z, dxp1/max(dxp1))
ax3.legend(['Wake', 'Wake kick'])

# ax3.set_xlim((-6, 3))
# ax3.set_xlim((-66, -57))
ax3.set_ylim(-2.1, 2.1)
ax3.set_xlabel(r'$z$ [m]', fontsize=22)
# ax3.legend(loc=0)

#### Only the first 3 bunches concatenated into a beam in a ring. Track for two turns using a shorter slicer and plot the wake kicks of the second turn.
---

In [19]:
slicer = UniformBinSlicer(500, z_cuts=(-5, 0.5))

bunches_source = sum(bunches_list[:3])
bunches_target = sum(bunches_list[3:])
times_s, ages_s, cmoments_s, betas_s = get_times_and_moments(bunches_source, slicer)
times_t, ages_t, cmoments_t, betas_t = get_times_and_moments(bunches_source, slicer)

bunches_target.dt = 0

n_turns, n_bunches = 2, 1
times_reg = np.zeros((n_turns, n_bunches, slicer.n_slices))
cmoments_reg = np.zeros((n_turns, n_bunches, slicer.n_slices))

times_reg[0,0,:] = times_t
times_reg[1,0,:] = times_s
cmoments_reg[0,0,:] = cmoments_t
cmoments_reg[1,0,:] = cmoments_s

ages = [0., T0]
betas = [bunches_target.beta, bunches_source.beta]

In [20]:
dxp_mt = convolve_PyHEADTAIL_mb(bunches_target, times_reg, ages, cmoments_reg, betas)
dxp_mt = np.squeeze(dxp_mt[0])
dxp_mt_np = convolve_multibunch(bunches, times_reg, ages, cmoments_reg, betas, wakefunction)
dxp_mt_np = np.squeeze(dxp_mt_np[0])

In [21]:
times = np.squeeze(times)
cmoments = np.squeeze(cmoments)
dxp1 = np.squeeze(delta_xp_mb)
dxp2 = np.squeeze(delta_xp_sb)
dxp3 = np.squeeze(delta_xp_np)

t0 = np.concatenate((times-times[-1], (times-times[0])[1:]))

col = sns.color_palette('viridis', 3, 0.8)
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(16,10), sharex=True)

ax1.plot(t0, wakefunction(t0)[::-1]/max(wakefunction(t0)[::-1]))
ax1.plot(times-times[-1], cmoments/max(cmoments))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Wake', 'Bunch'], loc=0)
ax1.set_xlim(-220e-9, 20e-9)

ax2.plot(times-times[-1], dxp1/max(dxp1))
ax2.plot(times-times[-1], dxp2/max(dxp2))
ax2.plot(times-times[-1], -dxp3/max(-dxp3))
ax2.legend(['PyHEADTAIL MB', 'PyHEADTAIL SB', 'Notebook manual'], loc=0)

ax3.plot(times-times[-1], 2*cmoments/max(cmoments))
ax3.plot(times-times[-1], dxp1/max(dxp1))
ax3.legend(['Wake', 'Wake kick'])

for i in range(n_turns):
    ax3.plot(times_reg[i,0,:]-times[-1]-ages[i],
             2*cmoments_reg[i,0,:]/max(cmoments_reg[1,0,:]),
             c=col[i], label='One bunch, turn {:d}'.format(i+1))
ax3.plot(times_reg[1,0,:]-times[-1]-ages[i], dxp_mt/max(dxp_mt))
ax3.plot(times_reg[1,0,:]-times[-1]-ages[i], -dxp_mt_np/max(-dxp_mt_np))

# ax3.set_xlim((-22e-9, 2e-9))
# ax3.set_xlim((-220e-9, -190e-9))

#### Multiple turns, multiple bunches ($2\times 3$) in a ring
---

In [22]:
slicer = UniformBinSlicer(100, z_cuts=(-.5, .5))

class CircularResonatorMod(CircularResonator):
    
    def get_wake_kicks(self, slicer):
        wake_kicks = []
        
        if self.Yokoya_X1:
            wake_function = self.function_transverse(self.Yokoya_X1)
            wake_kicks.append(ConstantWakeKickX(
                wake_function, slicer, self.n_turns_wake))
            
        return wake_kicks
    
wake = CircularResonatorMod(R_shunt=1e6, frequency=700e6, Q=350, n_turns_wake=4)

wakefields = WakeField(slicer, wake, circumference=circumference)
wakekick = wakefields.wake_kicks[0]
wakefunction = wakekick.wake_function

*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!


In [23]:
bunches_sublist = deepcopy(bunches_list[:3])
for i, b in enumerate(bunches_sublist):
    b.z += dt[i] * (b.beta*c)

wakefields.track(bunches_sublist)
times_pyht, ages_pyht, moments_pyht, betas_pyht = wakekick._extract_slice_set_data(wakefields.slice_set_deque)

dxp_pyht = wakekick.dxp
print times_pyht.shape

(1, 3, 100)


In [24]:
times = np.squeeze(times)
cmoments = np.squeeze(cmoments)
dxp1 = np.squeeze(delta_xp_mb)
dxp2 = np.squeeze(delta_xp_sb)
dxp3 = np.squeeze(delta_xp_np)

t0 = np.concatenate((times-times[-1], (times-times[0])[1:]))
to_z = beta*c

col = sns.color_palette('cubehelix', 3, 0.8)
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(16,10), sharex=True)

ax1.plot(t0*to_z, wakefunction(t0)[::-1]/max(wakefunction(t0)[::-1]))
ax1.plot((times-times[-1])*to_z, cmoments/max(cmoments))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Wake', 'Bunch'], loc=0)
ax1.set_xlim(-66, 6)

ax2.plot((times-times[-1])*to_z, dxp1/max(dxp1))
ax2.plot((times-times[-1])*to_z, dxp2/max(dxp2))
ax2.plot((times-times[-1])*to_z, -dxp3/max(-dxp3))
ax2.legend(['PyHEADTAIL MB', 'PyHEADTAIL SB', 'Notebook manual'], loc=0)

ax3.plot((times-times[-1])*to_z, 2*cmoments/max(cmoments))
ax3.plot((times-times[-1])*to_z, dxp1/max(dxp1))
ax3.legend(['Wake', 'Wake kick'])

    
k = times_pyht.shape[0]-1
scale = 1
if k>1:
    k=1
if k==0:
#     ax3.set_xlim((-6, 3))
    scale = max(dxp1[-500:])/max(dxp1)
if k==1:
#     ax3.set_xlim((-66, -57))
    scale = 1

for i in range(3):
    ax3.plot((times_pyht[k,i,:]-ages_pyht[k]-times[-1]-dt[i])*to_z,
             2*moments_pyht[0,i,:]/1./max(moments_pyht[0,2,:]), '--',
             c=col[i], label='PyHEADTAIL, bunch# {:d}'.format(i+1))
    ax3.plot((times_pyht[k,i,:]-ages_pyht[k]-times[-1]-dt[i])*to_z,
             dxp_pyht[i]/max(dxp_pyht[2]) * scale, lw=3,
             c=col[i])

fig.suptitle("Resulting individual wake kicks after {:d} turn(s)".format(k+1), fontsize=22)
ax3.set_ylim(-2.1, 2.1)
ax3.set_xlabel(r'$z$ [m]', fontsize=22)
ax3.legend(loc=0)

In [0]:
plt.close('all')